# Introduction
Find S&P outliers

In [2]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import pandas_datareader.data as web

# https://plot.ly/scikit-learn/plot_stock_market/
import plotly.plotly as py
import plotly.graph_objs as go

import datetime
import numpy as np
import matplotlib.pyplot as plt

from matplotlib.collections import LineCollection
from sklearn import cluster, covariance, manifold

import warnings
warnings.filterwarnings("ignore") 


In [3]:

def parse_html_table(table):
    n_columns = 0
    n_rows=0
    column_names = []

    # Find number of rows and columns
    # we also find the column titles if we can
    for row in table.find_all('tr'):
        
        # Determine the number of rows in the table
        td_tags = row.find_all('td')
        if len(td_tags) > 0:
            n_rows+=1
            if n_columns == 0:
                # Set the number of columns for our table
                n_columns = len(td_tags)
                
        # Handle column names if we find them
        th_tags = row.find_all('th') 
        if len(th_tags) > 0 and len(column_names) == 0:
            for th in th_tags:
                column_names.append(th.get_text().strip())

    # Safeguard on Column Titles
    if len(column_names) > 0 and len(column_names) != n_columns:
        raise Exception("Column titles do not match the number of columns")

    columns = column_names if len(column_names) > 0 else range(0,n_columns)
    df = pd.DataFrame(columns = columns,
                      index= range(0,n_rows))
    row_marker = 0
    for row in table.find_all('tr'):
        column_marker = 0
        columns = row.find_all('td')
        for column in columns:
            df.iat[row_marker,column_marker] = column.get_text().strip()
            column_marker += 1
        if len(columns) > 0:
            row_marker += 1
            
    # Convert to float if possible
    for col in df:
        try:
            df[col] = df[col].astype(float)
        except ValueError:
            pass
    
    return df

def getTickers(url, fname, column_name):
    r = requests.get(url)
    # Extract the content
    c = r.content
    #print(c)

    # Create a soup object
    soup = BeautifulSoup(c, 'lxml')

    # Find the element on the webpage
    htable = soup.find('table', {'class': 'wikitable sortable'})
    #print(htable)

    df = parse_html_table(htable)
    print(df.head())

    #df = df.sort_values(column_name)
    # add ticker column on the table
    #df['ticker'] = df[column_name]
    df.insert(0, 'ticker', df[column_name])
    dfcsv = df.iloc[:,[0,1,2]]

    #dfcsv.to_csv(fname, index=False)
    return df

def getDailyRet(s):
    #print(s)
    ohlc = web.DataReader(s, 'iex', start, end)
    ohlc['retd0'] = ohlc['close'].pct_change(1)
    ohlc['retd5'] = ohlc['close'].pct_change(5)
    ohlc['retd0hilo'] = ohlc['close']/ohlc['open'] - 1
    ohlc['sym'] = s
    
    return ohlc['retd0'].dropna()

def get(tickers, startdate, enddate):
    def data(ticker):
        #return (pdr.get_data_yahoo(ticker, start=startdate, end=enddate))
        return web.DataReader(s, 'iex', start, end)
    datas = map(data, tickers)
    return(pd.concat(datas, keys=tickers, names=['Ticker', 'Date']))

# all_data = get(tickers, stocks_start, stocks_end)

In [10]:
# Make the GET request to a url
DOW30 = "https://en.wikipedia.org/wiki/Dow_Jones_Industrial_Average"
SP500 = "http://en.wikipedia.org/wiki/List_of_S%26P_500_companies"

#index = getTickers(DOW30, 'csv/dow30.csv', 'Symbol')
index = getTickers(SP500, 'csv/sp500.csv', 'Symbol')

index['Symbol'] = index['Symbol'].str.replace('-','.')
index['ticker'] = index['ticker'].str.replace('-','.')

# remove WAB
wab = index[index['Symbol']=='WAB']
#print(np.array(wab.index))
index = index.drop(np.array(wab.index))

symbols = np.array(index['ticker']).T
names = np.array(index['ticker']).T
print(names)

              Security Symbol SEC filings             GICS Sector  \
0           3M Company    MMM     reports             Industrials   
1  Abbott Laboratories    ABT     reports             Health Care   
2          AbbVie Inc.   ABBV     reports             Health Care   
3          ABIOMED Inc   ABMD     reports             Health Care   
4        Accenture plc    ACN     reports  Information Technology   

                GICS Sub Industry    Headquarters Location  \
0        Industrial Conglomerates      St. Paul, Minnesota   
1           Health Care Equipment  North Chicago, Illinois   
2                 Pharmaceuticals  North Chicago, Illinois   
3           Health Care Equipment   Danvers, Massachusetts   
4  IT Consulting & Other Services          Dublin, Ireland   

  Date first added[3][4]        CIK      Founded  
0                           66740.0         1902  
1             1964-03-31     1800.0         1888  
2             2012-12-31  1551152.0  2013 (1888)  
3       

In [9]:
from datetime import datetime

start = datetime(2018, 12, 31)
end = datetime(2019, 3, 1)

data = []

for s in index['ticker'].unique():
    print(s)
    ohlc = web.DataReader(s, 'iex', start, end)
    ohlc['retd0'] = ohlc['close'].pct_change(1)
    ohlc['retd5'] = ohlc['close'].pct_change(5)
    ohlc['retd0hilo'] = ohlc['close']/ohlc['open'] - 1
    ohlc['sym'] = s
    data.append(ohlc)
    
print('loaded stocks daily returns.')

#prices["Return"] = prices.groupby("Ticker")["Price"].pct_change(1)
dfAll = pd.concat(data)
dfAll.tail()

MMM
ABT
ABBV
ABMD
ACN
ATVI
ADBE
AMD
AAP
AES
AMG
AFL
A
APD
AKAM
ALK
ALB
ARE
ALXN
ALGN
ALLE
AGN
ADS
LNT
ALL
GOOGL
GOOG
MO
AMZN
AEE
AAL
AEP
AXP
AIG
AMT
AWK
AMP
ABC
AME
AMGN
APH
APC
ADI
ANSS
ANTM
AON
AOS
APA
AIV
AAPL
AMAT
APTV
ADM
ARNC
ANET
AJG
AIZ
ATO
T
ADSK
ADP
AZO
AVB
AVY
BHGE
BLL
BAC
BK
BAX
BBT
BDX
BRK.B
BBY
BIIB
BLK
HRB
BA
BKNG
BWA
BXP
BSX
BHF
BMY
AVGO
BR
BF.B
CHRW
COG
CDNS
CPB
COF
CPRI
CAH
KMX
CCL
CAT
CBOE
CBRE
CBS
CE
CELG
CNC
CNP
CTL
CERN
CF
SCHW
CHTR
CVX
CMG
CB
CHD
CI
XEC
CINF
CTAS
CSCO
C
CFG
CTXS
CLX
CME
CMS
KO
CTSH
CL
CMCSA
CMA
CAG
CXO
COP
ED
STZ
COO
CPRT
GLW
COST
COTY
CCI
CSX
CMI
CVS
DHI
DHR
DRI
DVA
DE
DAL
XRAY
DVN
FANG
DLR
DFS
DISCA
DISCK
DISH
DG
DLTR
D
DOV
DWDP
DTE
DRE
DUK
DXC
ETFC
EMN
ETN
EBAY
ECL
EIX
EW
EA
EMR
ETR
EOG
EFX
EQIX
EQR
ESS
EL
EVRG
ES
RE
EXC
EXPE
EXPD
EXR
XOM
FFIV
FB
FAST
FRT
FDX
FIS
FITB
FE
FRC
FISV
FLT
FLIR
FLS
FLR
FMC
FL
F
FTNT
FTV
FBHS
BEN
FCX
GPS
GRMN
IT
GD
GE
GIS
GM
GPC
GILD
GPN
GS
GWW
HAL
HBI
HOG
HRS
HIG
HAS
HCA
HCP
HP
HSIC
HSY
HES
HPE
HLT
HFC
HOLX
HD
HON
H

,open,high,low,close,volume,retd0,retd5,retd0hilo,sym
date,,,,,,,,,
2019-02-25,95.00,95.180,94.2400,94.39,1295510,-0.002325,-0.010276,-0.006421,ZTS
2019-02-26,94.02,94.495,93.3362,93.49,1829388,-0.009535,-0.014962,-0.005637,ZTS
2019-02-27,93.34,94.160,93.1495,93.80,1535147,0.003316,-0.006251,0.004928,ZTS
2019-02-28,93.96,94.665,93.8400,94.23,2253418,0.004584,0.008131,0.002874,ZTS
2019-03-01,94.99,96.150,94.5100,95.75,2069346,0.016131,0.012049,0.008001,ZTS


In [11]:
print(dfAll.head())

df = dfAll[['sym', 'retd0']]
df.head()
df = df.reset_index().pivot(index='date', columns='sym', values='retd0').dropna()
df.head()
print(len(df.columns))

                open      high       low     close   volume     retd0  retd5  \
date                                                                           
2018-12-31  189.0331  190.3341  187.2058  189.2318  1804618       NaN    NaN   
2019-01-02  186.5304  189.6787  185.4181  189.6389  2475193  0.002151    NaN   
2019-01-03  186.9873  186.9873  181.6343  182.4983  3358241 -0.037654    NaN   
2019-01-04  185.4678  190.6619  184.7527  190.0064  2995052  0.041141    NaN   
2019-01-07  190.0461  190.9797  187.3647  189.5694  2162158 -0.002300    NaN   

            retd0hilo  sym  
date                        
2018-12-31   0.001051  MMM  
2019-01-02   0.016665  MMM  
2019-01-03  -0.024007  MMM  
2019-01-04   0.024471  MMM  
2019-01-07  -0.002508  MMM  
504


In [12]:
dfHiLo = dfAll[['sym', 'retd0hilo']]
dfHiLo.describe()
dfHiLo.head()
#variation = dfHiLo[np.isnan(dfHiLo['retd0hilo'])]
#variation

,sym,retd0hilo
date,,


In [13]:
#df.describe()
open = np.array([d.open for d in data]).astype(np.float)
close = np.array([d.close for d in data]).astype(np.float)

# The daily variations of the quotes are what carry most information
variation = close - open
print(variation)
print(len(variation))


[[ 0.1987  3.1085 -4.489  ... -0.57   -1.66   -1.32  ]
 [ 0.7565 -0.8859 -2.5084 ...  0.78    0.61    0.6   ]
 [ 0.5828 -1.9856 -2.3215 ... -0.73   -0.22    0.33  ]
 ...
 [ 0.68   -0.64   -1.27   ...  1.84   -0.28    0.49  ]
 [ 0.2485  1.5407 -0.1789 ...  0.54    0.     -0.27  ]
 [ 0.2695 -0.0299 -2.5051 ...  0.46    0.27    0.76  ]]
504


In [14]:
# variation
edge_model = covariance.GraphLassoCV()

# standardize the time series: using correlations rather than covariance
# is more efficient for structure recovery
X = variation.copy().T
X /= X.std(axis=0)
edge_model.fit(X)

print(edge_model)

GraphLassoCV(alphas=4, assume_centered=False, cv='warn', enet_tol=0.0001,
       max_iter=100, mode='cd', n_jobs=None, n_refinements=4, tol=0.0001,
       verbose=False)


In [15]:
_, labels = cluster.affinity_propagation(edge_model.covariance_)
n_labels = labels.max()

for i in range(n_labels + 1):
    print('Cluster %i: %s' % ((i + 1), ', '.join(names[labels == i])))

node_position_model = manifold.LocallyLinearEmbedding(
    n_components=2, eigen_solver='dense', n_neighbors=6)

embedding = node_position_model.fit_transform(X.T).T

Cluster 1: ABT, A, ADM, BDX, IFF, MTD, PKI, TFX, TMO, VAR, WAT
Cluster 2: ALLE
Cluster 3: AGN
Cluster 4: GOOGL, GOOG, AMZN, AAPL, MSFT, VRSN
Cluster 5: MO
Cluster 6: APH, PGR
Cluster 7: AON, AJG, MMC, WLTW
Cluster 8: AFL, AIZ
Cluster 9: ADBE, ADP, EL, FIS, FRC, GPN, JKHY, PAYX, CRM, SPGI, ZBH
Cluster 10: BLL
Cluster 11: BWA, IQV, MRO, ZTS
Cluster 12: ARE, BXP, CL, DRE, FRT, KIM, PLD, REG, SPG, SLG, VNO
Cluster 13: BSX, CTSH, EW, SYK
Cluster 14: AVGO, LKQ, LUV, TSCO
Cluster 15: CDNS, STZ, SYMC
Cluster 16: KMX, JBHT
Cluster 17: CBOE
Cluster 18: T, CBS, CTL, DISH
Cluster 19: CERN
Cluster 20: CF, SJM, K, MOS, NUE, PM
Cluster 21: SCHW, ETFC, RJF
Cluster 22: CMG
Cluster 23: CB, HIG, TRV
Cluster 24: BRK.B, CSCO, MSI, NDAQ
Cluster 25: CMCSA
Cluster 26: CPRT
Cluster 27: COST, NKE, SHW
Cluster 28: CVS
Cluster 29: ALGN, BAX, DHR, ECL, JNJ, MNST, SYY
Cluster 30: DRI, ULTA, WM
Cluster 31: DVA
Cluster 32: ALK, AAL, DAL, ORCL, SBUX, UAL
Cluster 33: XRAY
Cluster 34: CHTR, DISCA, DISCK, VIAB
Cluster 35

In [17]:
def matplotlib_to_plotly(cmap, pl_entries):
    h = 1.0/(pl_entries-1)
    pl_colorscale = []
    
    for k in range(pl_entries):
        C = list(map(np.uint8, np.array(cmap(k*h)[:3])*255))
        pl_colorscale.append([k*h, 'rgb'+str((C[0], C[1], C[2]))])
        
    return pl_colorscale

# Display a graph of the partial correlations
partial_correlations = edge_model.precision_.copy()
d = 1 / np.sqrt(np.diag(partial_correlations))
partial_correlations *= d
partial_correlations *= d[:, np.newaxis]
non_zero = (np.abs(np.triu(partial_correlations, k=1)) > 0.02)

# Plot the nodes using the coordinates of our embedding
trace = go.Scatter(x= embedding[0], y=embedding[1],
                   mode="markers", hoverinfo='none',
                   marker = dict(size=10,
                   color = np.random.randn(500),
                   #colorscale= matplotlib_to_plotly(plt.cm.spectral,500),))
                   colorscale= matplotlib_to_plotly(plt.cm.Spectral,500),))

# Plot the edges
start_idx, end_idx = np.where(non_zero)
#a sequence of (*line0*, *line1*, *line2*), where::
#            linen = (x0, y0), (x1, y1), ... (xm, ym)
segments = [[embedding[:, start], embedding[:, stop]]
            for start, stop in zip(start_idx, end_idx)]
values = np.abs(partial_correlations[non_zero])
trace1 = []
color=[]

for i in range(len(segments)):
    trace1_data = go.Scatter(
            x=[segments[i][0][0],segments[i][1][0]],
            y=[segments[i][0][1],segments[i][1][1]],
            mode = 'lines',
             hoverinfo='none',
            line = dict(
             color = "yellow", 
             width= 15*values[i]
        ))
    trace1.append(trace1_data)

# Add a label to each node. The challenge here is that we want to
# position the labels to avoid overlap with other labels
annotations= []

for index, (name, label, (x, y)) in enumerate(
        zip(names, labels, embedding.T)):

    dx = x - embedding[0]
    dx[index] = 1
    dy = y - embedding[1]
    dy[index] = 1
    this_dx = dx[np.argmin(np.abs(dy))]
    this_dy = dy[np.argmin(np.abs(dx))]
    if this_dx > 0:
        horizontalalignment = 'left'
        x = x + .002
    else:
        horizontalalignment = 'right'
        x = x - .002
    if this_dy > 0:
        verticalalignment = 'bottom'
        y = y + .002
    else:
        verticalalignment = 'top',
        y = y - .002
    annot = dict(x=x, y=y, text=name,showarrow=False
                )
    annotations.append(annot)

layout = go.Layout(
            showlegend=False,
            yaxis=dict(
                    zeroline=False, showticklabels=False,
                    range=[embedding[1].min() - .03 * embedding[1].ptp(),
                           embedding[1].max() + .03 * embedding[1].ptp()],
                    showgrid=False),
            xaxis=dict(
                    zeroline=False, showticklabels=False,
                    range=[embedding[0].min() - .15 * embedding[0].ptp(),
                           embedding[0].max() + .10 * embedding[0].ptp(),],
                    showgrid=False),
            annotations=annotations,
            height=900)

trace1.append(trace)

fig = go.Figure(data= trace1, layout= layout)

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
    
iplot(fig)

In [18]:
for i in range(n_labels + 1):
    print('Cluster %i: %s' % ((i + 1), ', '.join(names[labels == i])))


Cluster 1: ABT, A, ADM, BDX, IFF, MTD, PKI, TFX, TMO, VAR, WAT
Cluster 2: ALLE
Cluster 3: AGN
Cluster 4: GOOGL, GOOG, AMZN, AAPL, MSFT, VRSN
Cluster 5: MO
Cluster 6: APH, PGR
Cluster 7: AON, AJG, MMC, WLTW
Cluster 8: AFL, AIZ
Cluster 9: ADBE, ADP, EL, FIS, FRC, GPN, JKHY, PAYX, CRM, SPGI, ZBH
Cluster 10: BLL
Cluster 11: BWA, IQV, MRO, ZTS
Cluster 12: ARE, BXP, CL, DRE, FRT, KIM, PLD, REG, SPG, SLG, VNO
Cluster 13: BSX, CTSH, EW, SYK
Cluster 14: AVGO, LKQ, LUV, TSCO
Cluster 15: CDNS, STZ, SYMC
Cluster 16: KMX, JBHT
Cluster 17: CBOE
Cluster 18: T, CBS, CTL, DISH
Cluster 19: CERN
Cluster 20: CF, SJM, K, MOS, NUE, PM
Cluster 21: SCHW, ETFC, RJF
Cluster 22: CMG
Cluster 23: CB, HIG, TRV
Cluster 24: BRK.B, CSCO, MSI, NDAQ
Cluster 25: CMCSA
Cluster 26: CPRT
Cluster 27: COST, NKE, SHW
Cluster 28: CVS
Cluster 29: ALGN, BAX, DHR, ECL, JNJ, MNST, SYY
Cluster 30: DRI, ULTA, WM
Cluster 31: DVA
Cluster 32: ALK, AAL, DAL, ORCL, SBUX, UAL
Cluster 33: XRAY
Cluster 34: CHTR, DISCA, DISCK, VIAB
Cluster 35

In [19]:
df.describe()

sym,A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,...,XEL,XLNX,XOM,XRAY,XRX,XYL,YUM,ZBH,ZION,ZTS
count,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000,...,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000
mean,0.004636,0.002279,0.000679,0.002907,-0.002937,0.002897,0.001990,0.002276,0.003782,0.003878,...,0.002692,0.010300,0.004238,0.007198,0.011486,0.003484,0.001049,0.004756,0.005783,0.002918
std,0.013740,0.027218,0.019939,0.023294,0.019568,0.015691,0.025606,0.016667,0.011220,0.014713,...,0.009184,0.033208,0.011568,0.029828,0.021344,0.014520,0.009097,0.015416,0.011182,0.015503
min,-0.036840,-0.074507,-0.048804,-0.099607,-0.062180,-0.017366,-0.058875,-0.047195,-0.034142,-0.039498,...,-0.021311,-0.037652,-0.015353,-0.026232,-0.021858,-0.034279,-0.025153,-0.022425,-0.012513,-0.034565
25%,-0.002977,-0.015448,-0.008517,-0.005639,-0.009385,-0.007947,-0.010738,-0.005512,-0.001735,-0.002398,...,-0.001781,-0.007077,-0.004366,-0.004331,-0.000318,-0.004483,-0.002219,-0.004033,-0.003518,-0.005479
50%,0.006239,0.001379,0.001219,0.003098,0.001255,0.002192,0.001452,0.005050,0.003473,0.001125,...,0.001683,0.008516,0.003790,0.003337,0.008127,0.003509,0.001402,0.004390,0.005305,0.003316
75%,0.013013,0.014580,0.014525,0.010511,0.008731,0.008120,0.011850,0.012762,0.008919,0.011533,...,0.008641,0.017222,0.010297,0.012609,0.015528,0.012339,0.005572,0.011619,0.011057,0.011374
max,0.034614,0.065868,0.037591,0.068334,0.032217,0.061632,0.056466,0.032225,0.038884,0.048632,...,0.028280,0.184367,0.036869,0.176485,0.113992,0.042191,0.026026,0.064257,0.033957,0.056771


In [21]:
cumRet

,Ticker,TotalReturn
271,KHC,-0.233821
236,HSIC,-0.221938
298,M,-0.169804
124,CTL,-0.135244
448,TTWO,-0.129282
4,ABBV,-0.120409
127,CVS,-0.103638
351,NWL,-0.093927
399,RMD,-0.069672
52,ATVI,-0.059474


In [28]:
# plot the return - sorted bar chart
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

cumRet = pd.DataFrame([[s, df[s].sum()] for s in df.columns ], columns=['Ticker', 'TotalReturn']).sort_values(by='TotalReturn', ascending=True)

spyRet = np.array(getDailyRet('SPY'))
spyStd = np.std(spyRet)

up=spyCum+5*spyStd
down=spyCum-5*spyStd

# show only <.1 and >.9 in percentile
df2b = cumRet[(cumRet.TotalReturn < np.percentile(cumRet.TotalReturn,10))]
df2u = cumRet[(cumRet.TotalReturn > np.percentile(cumRet.TotalReturn,90))]

df2merged = pd.concat([df2b, df2u])

spyCum = np.empty(len(df2merged))
spyCum.fill(np.sum(spyRet))

#trace = go.Bar(x=df3['TotalReturn'], y=df3['Symbol'], orientation='h', )
trace = go.Bar(x=df2merged['Ticker'], y=df2merged['TotalReturn'], name='Stocks')
trace0 = go.Scatter(x=df2merged['Ticker'], y=spyCum, name='Index')

trace02stdhigh = go.Scatter(y=up, x=df2merged['Ticker'], name='Index+5 Std')
trace02stdlow = go.Scatter(y=down, x=df2merged['Ticker'], name='Index-5 Std')

data2=[trace, trace0, trace02stdhigh, trace02stdlow]

iplot(data2)